[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_)


In [3]:
from pyfian.time_value.irr import irr

irr([-1000, 300, 420, 680, 900], guess=0.1)



0.35000880649398824